<a href="https://colab.research.google.com/github/NataliaTarasovaNatoshir/NataliaTarasovaNatoshir/blob/main/heron_data_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import json
import pandas as pd

# Explore the data

In [5]:
with open("/content/example.json") as file:
  data_json = json.load(file)

In [13]:
data_json.keys()

dict_keys(['transactions'])

In [26]:
data = pd.DataFrame.from_dict(data_json['transactions'])

In [22]:
len(data)

27

In [23]:
data

,description,amount,date
0,Spotify,-14.99,2021-01-29
1,Spotify,-14.99,2020-12-29
2,Spotify,-14.99,2020-11-29
3,Spotify,-14.99,2020-10-29
4,Netflix,-20.00,2020-02-15
5,Netflix,-20.00,2020-03-14
6,Netflix,-20.00,2020-04-16
7,Netflix,-20.00,2020-05-15
8,Aug2020 Acme Corp Salary,4000.00,2020-08-01
9,Sep2020 Acme Corp Salary,4000.00,2020-09-01


# Transactions with the same name, amount and date diff between transactions. At least 2 transactions per group

In [27]:
candidates = data.groupby(['description', 'amount']).aggregate({'description': len})
candidates.columns = ['trans_num']
candidates = candidates[candidates['trans_num'] > 1].reset_index()
candidates['group_name'] = candidates[['description', 'amount']].apply(lambda x: x[0] + '_' + str(x[1]), axis=1)
candidates.head()

,description,amount,trans_num,group_name
0,Netflix,-20.00,4,Netflix_-20.0
1,Spotify,-14.99,4,Spotify_-14.99


In [28]:
data = pd.merge(data, candidates, how='left', on=['description', 'amount'])
data

,description,amount,date,trans_num,group_name
0,Spotify,-14.99,2021-01-29,4.0,Spotify_-14.99
1,Spotify,-14.99,2020-12-29,4.0,Spotify_-14.99
2,Spotify,-14.99,2020-11-29,4.0,Spotify_-14.99
3,Spotify,-14.99,2020-10-29,4.0,Spotify_-14.99
4,Netflix,-20.00,2020-02-15,4.0,Netflix_-20.0
5,Netflix,-20.00,2020-03-14,4.0,Netflix_-20.0
6,Netflix,-20.00,2020-04-16,4.0,Netflix_-20.0
7,Netflix,-20.00,2020-05-15,4.0,Netflix_-20.0
8,Aug2020 Acme Corp Salary,4000.00,2020-08-01,NaN,NaN
9,Sep2020 Acme Corp Salary,4000.00,2020-09-01,NaN,NaN


# Same amount, similar name, special words

In [62]:
special_words = ['salary', 'payslip', 'lunch']

In [65]:
data['descriptions_words'] = data['description'].apply(lambda x: x.split())
data['descriptions_words'] = data['descriptions_words'].apply(lambda x: [el.lower().replace(':', '') for el in x])
data['special_words'] = data['descriptions_words'].apply(lambda x: "_".join(sorted([el for el in x if el in special_words])))

In [66]:
data

,description,amount,date,trans_num,group_name,descriptions_words,special_words
0,Spotify,-14.99,2021-01-29,4.0,Spotify_-14.99,[spotify],
1,Spotify,-14.99,2020-12-29,4.0,Spotify_-14.99,[spotify],
2,Spotify,-14.99,2020-11-29,4.0,Spotify_-14.99,[spotify],
3,Spotify,-14.99,2020-10-29,4.0,Spotify_-14.99,[spotify],
4,Netflix,-20.00,2020-02-15,4.0,Netflix_-20.0,[netflix],
5,Netflix,-20.00,2020-03-14,4.0,Netflix_-20.0,[netflix],
6,Netflix,-20.00,2020-04-16,4.0,Netflix_-20.0,[netflix],
7,Netflix,-20.00,2020-05-15,4.0,Netflix_-20.0,[netflix],
8,Aug2020 Acme Corp Salary,4000.00,2020-08-01,NaN,NaN,"[aug2020, acme, corp, salary]",salary
9,Sep2020 Acme Corp Salary,4000.00,2020-09-01,NaN,NaN,"[sep2020, acme, corp, salary]",salary


In [73]:
candidates = data[(data['group_name'].isnull() == True) & (data['special_words'].apply(len) > 0)].groupby(['amount', 'special_words']).aggregate({'description': len})
candidates.columns = ['trans_num_x']
candidates = candidates.reset_index()
candidates['group_name_x'] = candidates[['special_words', 'amount']].apply(lambda x: x[0] + '_' + str(x[1]), axis=1)
candidates

,amount,special_words,trans_num_x,group_name_x
0,5.0,lunch,6,lunch_5.0
1,4000.0,salary,5,salary_4000.0
2,5000.0,salary,4,salary_5000.0


In [74]:
data = pd.merge(data, candidates, how='left', on=['special_words', 'amount'])
for col in ('trans_num', 'group_name'):
  data.loc[data[col].isnull(), col] = data.loc[data[col].isnull(), col + '_x']

In [77]:
data

,description,amount,date,trans_num,group_name,descriptions_words,special_words,trans_num_x,group_name_x
0,Spotify,-14.99,2021-01-29,4.0,Spotify_-14.99,[spotify],,NaN,NaN
1,Spotify,-14.99,2020-12-29,4.0,Spotify_-14.99,[spotify],,NaN,NaN
2,Spotify,-14.99,2020-11-29,4.0,Spotify_-14.99,[spotify],,NaN,NaN
3,Spotify,-14.99,2020-10-29,4.0,Spotify_-14.99,[spotify],,NaN,NaN
4,Netflix,-20.00,2020-02-15,4.0,Netflix_-20.0,[netflix],,NaN,NaN
5,Netflix,-20.00,2020-03-14,4.0,Netflix_-20.0,[netflix],,NaN,NaN
6,Netflix,-20.00,2020-04-16,4.0,Netflix_-20.0,[netflix],,NaN,NaN
7,Netflix,-20.00,2020-05-15,4.0,Netflix_-20.0,[netflix],,NaN,NaN
8,Aug2020 Acme Corp Salary,4000.00,2020-08-01,5.0,salary_4000.0,"[aug2020, acme, corp, salary]",salary,5.0,salary_4000.0
9,Sep2020 Acme Corp Salary,4000.00,2020-09-01,5.0,salary_4000.0,"[sep2020, acme, corp, salary]",salary,5.0,salary_4000.0
